Add distance (to the center of Hanoi) and segment the projects

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np

# Replace with your file path
df = pd.read_excel("Real_estate_projects_Hanoi.xlsx")

# Quick peek
df.head()

,id,name,link,type,area range(m2),price range,location,contact,latitude,longitude,area_min,area_max,price_min,price_max,area_avg,price_avg,unit_price,district
0,13354,Căn hộ chung cư Hacinco Nguyễn Xiển,https://cafeland.vn/du-an/hacinco-nguyen-xien-...,Căn hộ chung cư,58 - 103,1102 - 1957,https://nhadat.cafeland.vn/map/tin-du-an/20.97...,/newsletter/3-4219/,20.977536,105.814116,58.0,103.0,1102.0,1957.0,80.5,1529.5,19.0,Quận Hoàng Mai
1,13350,Căn hộ chung cư Green Tower,https://cafeland.vn/du-an/green-tower-du-an-nh...,Căn hộ chung cư,56.8 - 82,1136 - 1640,https://nhadat.cafeland.vn/map/tin-du-an/20.99...,/newsletter/3-4208/,20.991173,105.760270,56.8,82.0,1136.0,1640.0,69.4,1388.0,20.0,Quận Nam Từ Liêm
2,13332,Nhà Phố - Biệt Thự HCCI Hoàng Liệt,https://cafeland.vn/du-an/hcci-hoang-liet-du-a...,Nhà Phố - Biệt Thự,128 - 100,20480 - 16000,https://nhadat.cafeland.vn/map/tin-du-an/20.96...,/newsletter/3-4152/,20.966890,105.838003,128.0,100.0,20480.0,16000.0,114.0,18240.0,160.0,Quận Hoàng Mai
3,13329,Khu Đô Thị Palmy Biztown,https://cafeland.vn/du-an/palmy-biztown-du-an-...,Khu Đô Thị,75 - 90,5025 - 6030,https://nhadat.cafeland.vn/map/tin-du-an/20.96...,/newsletter/3-4139/,20.967420,105.814353,75.0,90.0,5025.0,6030.0,82.5,5527.5,67.0,Huyện Thanh Trì
4,13317,Căn hộ chung cư CT3 Nghĩa Đô,https://cafeland.vn/du-an/ct3-nghia-do-du-an-c...,Căn hộ chung cư,43 - 69,1720 - 2760,https://nhadat.cafeland.vn/map/tin-du-an/21.05...,/newsletter/3-4122/,21.050124,105.793117,43.0,69.0,1720.0,2760.0,56.0,2240.0,40.0,Quận Bắc Từ Liêm


In [ ]:
#Add fixed CBD coordinates

CBD_LAT = 21.028511   # Hoàn Kiếm Lake fountain
CBD_LON = 105.852014

In [ ]:
#Define the Haversine function
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2=CBD_LAT, lon2=CBD_LON):
    R = 6371.0               # Earth radius (km)
    # convert to radian
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c             # distance in km

In [ ]:
df.loc[df["latitude"].isna() | df["longitude"].isna(), "dist_cbd_km"] = np.nan


In [ ]:
#Vectorise & add new column
df["dist_cbd_km"] = df.apply(
    lambda r: haversine(r["latitude"], r["longitude"]), axis=1
)

Price‑Tier Tagging

In [ ]:
bins  = [0, 35, 150, np.inf]
labels = ["Affordable", "Mid", "Luxury"]

df["price_tier"] = pd.cut(df["unit_price"], bins=bins, labels=labels, right=True)

# If unit_price sometimes stores text like "75 - 90", parse it into a numeric first
# e.g. take the mean of split range, then convert to float

In [ ]:
df["price_tier"].value_counts()


,count
price_tier,
Affordable,191
Mid,107
Luxury,12


In [ ]:
df.to_excel("Real_estate_projects_Hanoi_enriched.xlsx", index=False)


In [ ]:
from google.colab import files
files.download("Real_estate_projects_Hanoi_enriched.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Data cleaning**
Load & rename columns
Drop duplicates
Parse “range” text → min / max / avg
Swap reversed min–max pairs
Re-compute derived metrics (unit_price, price_tier)
Handle missing numeric values
Validate lat/long inside Hà Nội bounds
Standardise district and type labels
Create township & outlier flags
Export clean file (single source of truth)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd, numpy as np, re

df_raw = pd.read_excel("Real_estate_Hanoi.xlsx")
print(df_raw.shape)        # (364, 20)  ← in the sample file
df_raw.head(3)

(364, 20)


,id,name,link,type,area range(m2),price range,location,contact,latitude,longitude,area_min,area_max,price_min,price_max,area_avg,price_avg,unit_price,district,dist_cbd_km,price_tier
0,13354,Căn hộ chung cư Hacinco Nguyễn Xiển,https://cafeland.vn/du-an/hacinco-nguyen-xien-...,Căn hộ chung cư,58 - 103,1102 - 1957,https://nhadat.cafeland.vn/map/tin-du-an/20.97...,/newsletter/3-4219/,20.977536,105.814116,58.0,103.0,1102.0,1957.0,80.5,1529.5,19.0,Quận Hoàng Mai,6.899656,Affordable
1,13350,Căn hộ chung cư Green Tower,https://cafeland.vn/du-an/green-tower-du-an-nh...,Căn hộ chung cư,56.8 - 82,1136 - 1640,https://nhadat.cafeland.vn/map/tin-du-an/20.99...,/newsletter/3-4208/,20.991173,105.760270,56.8,82.0,1136.0,1640.0,69.4,1388.0,20.0,Quận Nam Từ Liêm,10.388933,Affordable
2,13332,Nhà Phố - Biệt Thự HCCI Hoàng Liệt,https://cafeland.vn/du-an/hcci-hoang-liet-du-a...,Nhà Phố - Biệt Thự,128 - 100,20480 - 16000,https://nhadat.cafeland.vn/map/tin-du-an/20.96...,/newsletter/3-4152/,20.966890,105.838003,128.0,100.0,20480.0,16000.0,114.0,18240.0,160.0,Quận Hoàng Mai,7.004652,Luxury


In [ ]:
#Standardise Column Names
df = df_raw.copy()
df.columns = (
    df.columns
      .str.strip()            # trim spaces
      .str.lower()
      .str.replace(r"\s+", "_", regex=True)
      .str.replace(r"[()]", "", regex=True)
)

In [ ]:
#Remove Perfect Duplicates
before = df.shape[0]
df = df.drop_duplicates(subset=["id"])     # OR ["name","latitude","longitude"]
print("Dropped", before - df.shape[0], "duplicate rows")

Dropped 0 duplicate rows


In [ ]:
mask = df["area_min"] > df["area_max"]
df.loc[mask, ["area_min","area_max"]] = df.loc[mask, ["area_max","area_min"]].values

mask = df["price_min"] > df["price_max"]
df.loc[mask, ["price_min","price_max"]] = df.loc[mask, ["price_max","price_min"]].values

# Re-compute averages for swapped rows
for col in ["area_avg","price_avg"]:
    df[col] = df[[col.replace("avg","min"), col.replace("avg","max")]].mean(axis=1)

In [ ]:
#Numeric blanks → NaN are already handled; decide imputation vs drop
num_cols = ["area_min","area_max","price_min","price_max","unit_price",
            "latitude","longitude"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# 6-b  Drop rows with no price information (rare in this file)
df = df.dropna(subset=["unit_price"])

In [ ]:
# keep only points inside Vietnam bounding box
coord_mask = (
    (df.latitude.between(20.5, 21.5)) &     # lat range around Hà Nội
    (df.longitude.between(105.3, 106.2))
)
df = df[coord_mask]

In [ ]:
# District names: remove prefixes, accents optional
df["district"] = (
    df["district"]
      .str.strip()
      .str.title()
      .str.replace("Tp\\.?", "Thành Phố", regex=True)
      .str.replace("Quan", "Quận", regex=False)
      .str.replace("Huyen", "Huyện", regex=False)
)

# Project type buckets                                  ↓ customise list
mapping = {"Căn Hộ Chung Cư":"Căn hộ chung cư",
           "Khu Đô Thị":"Khu Đô Thị",
           "Nhà Phố - Biệt Thự":"Nhà Phố – Biệt Thự"}
df["type_std"] = df["type"].map(mapping).fillna("Khác")


In [ ]:
# Z-score on unit_price
z = (df["unit_price"] - df["unit_price"].mean()) / df["unit_price"].std()
df["outlier_flag"] = (abs(z) > 3)

In [ ]:
df.to_excel("Real_estate_Hanoi_clean.xlsx", index=False)


In [ ]:
from google.colab import files
files.download("Real_estate_Hanoi_clean.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>